In [1]:
# %pip install -q git+https://github.com/huggingface/transformers.git

In [2]:
%pip install -q datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install captum

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [5]:
from PIL import Image
from torchvision import transforms
import pandas as pd
import os
import csv
import json

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image

import torch
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from transformers import AutoProcessor
from transformers import AutoModelForCausalLM


from captum.attr import TokenReferenceBase, configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [7]:
import os
import pandas as pd
clef2022_valid_path = '/hdd2/datasets/ImageClef2022medCaption/clef2022/Valid'
test_df = pd.read_csv(os.path.join(clef2022_valid_path, 'caption_prediction_valid.csv'),sep = '\t')
test_df['ID'] = test_df['ID'].apply(lambda x: os.path.join(clef2022_valid_path, 'valid', x + '.jpg'))
test_df.reset_index(inplace = True, drop = True)

test_df.head()
test_df_images = test_df['ID']
csv_path2 = "/hdd2/datasets/ImageClef2022medCaption/clef2022/Valid/test_results/blip_pred_result_80_5.csv"
pred_df = pd.read_csv(os.path.join(csv_path2),sep = '\t')
prediction_captions = pred_df['pred']

In [8]:
from transformers import AutoProcessor, BlipForConditionalGeneration, AutoTokenizer
from transformers import AutoModelForCausalLM
import torch
from captum.attr import IntegratedGradients, Saliency
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F




output_dir = "/hdd2/datasets/ImageClef2022medCaption/clef2022/blip_weights"
processor = AutoProcessor.from_pretrained(output_dir)
model = BlipForConditionalGeneration.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model.eval()

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [10]:
class MultimodalModel(torch.nn.Module):
    def __init__(self, cap_model, processor):
        super().__init__()
        self.cap_model = cap_model
        self.processor = processor

    def forward(self, input_image, input_text):
        outputs = model(input_ids=input_text,
                pixel_values=input_image,
                )            
        #generated_ids = self.cap_model.generate(pixel_values=input_image, input_ids=input_text, max_length=1)     
        # compute token_ll and that will be passed to attribute function

        logits = outputs.logits[:, :-1, :]

        targets = input_text[:, 1:]
 
        tokens_nll = F.cross_entropy(
        logits.view(-1, logits.size(-1)), 
        targets.view(-1), reduction='none')

        
        return tokens_nll

In [11]:
# Initialize the multimodal model
multimodal_model = MultimodalModel(model, processor)

# Initialize the Integrated Gradients attribution algorithm
ig = IntegratedGradients(multimodal_model)

In [12]:
img_path = test_df_images[0]
image = Image.open(img_path)
img = image.resize((224, 224), resample=Image.BILINEAR)
pixel_values = processor(images=img, return_tensors="pt").pixel_values
#pixel_values = torch.tensor(pixel_values, requires_grad=True)
pixel_values = pixel_values.detach().numpy()
pixel_values = torch.tensor(pixel_values, requires_grad=True)
generated_caption = prediction_captions[0]
# Tokenize the target caption
caption_ids = tokenizer(generated_caption, add_special_tokens=True, return_tensors='pt').input_ids
print(caption_ids)

tensor([[  101,  1060,  9447, 28711,  8560,  2265,  9808,  2618,  4747, 21252,
          4649,  3258,   102]])


In [13]:
input_ids = torch.tensor([[101,  1060,  9447]])

In [14]:
outputs = model(input_ids=input_ids,
                pixel_values=pixel_values,
                )

predicted_token_indices = outputs.logits

# output_sh = outputs.logits[:, :-1, :]
# print(output_sh)

tensor([[[-2.4079, -4.0878, -4.0878,  ..., -4.0878, -4.0870, -4.0870],
         [-0.1993, -0.7721, -0.7721,  ..., -0.7721, -0.7721, -0.7721]]],
       grad_fn=<SliceBackward0>)


In [25]:
# Iterate over the length of the original tensor
for i in range(caption_ids.shape[1]-1):
    # Token ID for [CLS]
    cls_token_id = 101
    # Create a subsequence up to the i-th element
    subsequence = caption_ids[:, :i + 1]
    #subsequence = subsequence.squeeze().detach().numpy()
    target_subsequence = caption_ids[:, :i + 2]
    target = target_subsequence[0, -1].item()
    # Compute the attribution scores for the current word
    #input = torch.randn(2, 3, 32, 32, requires_grad=True)
    #(img_attr,text_attr), _ = ig.attribute(inputs=(pixel_values, subsequence), baselines=(torch.zeros_like(pixel_values),torch.zeros_like(subsequence)), return_convergence_delta=True)
    (img_attr,text_attr), _ = ig.attribute(inputs=(pixel_values, subsequence))
    #(img_attr,text_attr), _ = ig.attribute(inputs=(pixel_values, subsequence), baselines=(torch.zeros_like(pixel_values),torch.zeros_like(subsequence)), target=target, return_convergence_delta=True)
    #img_attr, _ = ig.attribute(inputs=pixel_values, baselines=torch.zeros_like(pixel_values), target=caption_ids , return_convergence_delta=True)
    

    print(subsequence)
    #print(target_subsequence)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)